In [ ]:
import numpy as np
import sys
sys.path.append('../')
from cycada.data.data_loader import get_dataset
import PIL.Image as Image
from os.path import join
import os

In [ ]:
%cd  scripts
!ls

In [ ]:
import os
from PIL import Image
import random


def load_vocabulary(file_path):
    with open(file_path, 'r') as file:
        vocabulary = [line.strip() for line in file]

    word_to_index = {word: index for index, word in enumerate(vocabulary)}
    word_to_index["###"] = len(word_to_index)  # Handle special cases

    return word_to_index

def word_to_label(word, word_to_index):
    return word_to_index.get(word, 1)  # Returns the index for "###" if the word is not found

def generate_non_text_boxes(text_boxes, img_width, img_height, num_boxes):
    non_text_boxes = []
    while len(non_text_boxes) < num_boxes:
        valid_box = False
        while not valid_box:
            # Ensure random width and height are less than image dimensions
            max_width = min(300, img_width - 1)
            max_height = min(300, img_height - 1)

            width = random.randint(20, max_width)
            height = random.randint(20, max_height)

            left = random.randint(0, img_width - width)
            top = random.randint(0, img_height - height)
            right = left + width
            bottom = top + height
            box = (left, top, right, bottom)

            if not any(is_overlap(box, text_box) for text_box in text_boxes):
                valid_box = True

        non_text_boxes.append(box)

    return non_text_boxes


def is_overlap(box1, box2):
    left1, top1, right1, bottom1 = box1
    left2, top2, right2, bottom2 = box2
    return not (right1 < left2 or right2 < left1 or bottom1 < top2 or bottom2 < top1)

# Load vocabulary and process datasets as before

def process_dataset(gt_folder, img_folder, output_folder, word_to_index, numeric_naming=False, diagonal_points=False, generate_non_text=False):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images_output_folder = os.path.join(output_folder, 'images')
    if not os.path.exists(images_output_folder):
        os.makedirs(images_output_folder)

    labels_file = os.path.join(output_folder, 'labels.txt')
    
    with open(labels_file, 'w') as labels_f:
        print(labels_f)
        img_index = 0

        for gt_file in os.listdir(gt_folder):

            text_boxes = []  # List to store text box coordinates

            gt_path = os.path.join(gt_folder, gt_file)
            print(gt_path)

            if numeric_naming:
                # Extract the numeric part for numeric naming convention
                numeric_part = gt_file.split('_')[1].split('.')[0]  # Extracts the number from '100.txt'
                img_name = f'{numeric_part}.jpg'  # Constructs the corresponding image filename
            else:
                # Extract the numeric part for 'img_X.jpg' naming convention
                numeric_part = gt_file.split('_')[2].split('.')[0]
                img_name = f'img_{numeric_part}.jpg'

            img_path = os.path.join(img_folder, img_name)
            print(img_path)

            if os.path.exists(img_path):
                img = Image.open(img_path)
                try:
                    with open(gt_path, 'r', encoding='utf-8') as f:
                        lines = f.readlines()
                except UnicodeDecodeError:
                    with open(gt_path, 'r', encoding='ISO-8859-1') as f:
                        lines = f.readlines()

                for line in lines:

                    if ',' in line:
                        parts = line.strip().split(',')
                        print (parts)
                    else:
                        parts = line.strip().split(' ')
                        print (parts)
                   
                    if ((len(parts) == 9) or (len(parts) == 5)):
                       
                        if (len(parts) == 5):
                            try:
                                coordinates = list(map(int, parts[:4]))
                                label_word = parts[4].upper()
                                print(label_word)
                                label_index = word_to_label(label_word, word_to_index)  # Convert word to label index 
                                print(label_index)

                                if (label_index == 87623):
                                    label_index=0
                                else:
                                    label_index=1
                            except ValueError:
                                continue
                
                            # If coordinates are diagonal points (x1, y1, x3, y3)
                            x1, y1, x3, y3 = coordinates[0], coordinates[1], coordinates[2], coordinates[3]
                            left, top = min(x1, x3), min(y1, y3)
                            right, bottom = max(x1, x3), max(y1, y3)
                            text_boxes.append((left, top, right, bottom))
                        else:
                            try:
                                coordinates = list(map(int, parts[:8]))
                                label_word = parts[8]
                                print (label_word)
                                
                                label_index = word_to_label(label_word, word_to_index)  # Convert word to label index 
                                
                               # label_index=1
                                if (label_index == 87623):
                                    label_index=0
                                else:
                                    label_index=1

                            except ValueError:
                                continue
                            # Regular bounding box coordinates
                            left, top, right, bottom = min(coordinates[0::2]), min(coordinates[1::2]), max(coordinates[0::2]), max(coordinates[1::2])
                            text_boxes.append((left, top, right, bottom))
                                              # Generate non-text boxes
                        img_width, img_height = img.size
                        non_text_boxes = generate_non_text_boxes(text_boxes, img_width, img_height, len(text_boxes))

                        # Define bounding box and crop image
                        cropped_img = img.crop((left, top, right, bottom))
                        cropped_img_path = os.path.join(images_output_folder, f'{img_index}.png')
                        cropped_img.save(cropped_img_path)
                        print(cropped_img_path)

                        # Write label index
                        labels_f.write(f'{img_index} {label_index}\n')
                        img_index += 1
                        
                if generate_non_text:
                    # Generate random non-text boxes
                    img_width, img_height = img.size
                    non_text_boxes = generate_non_text_boxes(text_boxes, img_width, img_height, len(text_boxes))

                    for box in non_text_boxes:
                        left, top, right, bottom = box
                        cropped_img = img.crop((left, top, right, bottom))
                        cropped_img_path = os.path.join(images_output_folder, f'{img_index}.png')
                        cropped_img.save(cropped_img_path)
                        labels_f.write(f'{img_index} 0\n')  # Label for non-text boxes is 0
                        img_index += 1  
 # Load vocabulary
vocabulary_file = '../GenericVocabulary.txt'  # Replace with the path to your vocabulary file
word_to_index = load_vocabulary(vocabulary_file)

# Process each dataset
process_dataset('../2013/Challenge2_Training_Task1_GT', '../2013/Challenge2_Training_Task12_Images', '../FY/A-Aug-B-noaug/train/trainA', word_to_index, numeric_naming=True, diagonal_points=True, generate_non_text= True)
process_dataset('../2013/Challenge2_Test_Task1_GT', '../2013/Challenge2_Test_Task12_Images', '../FY/A-Aug-B-noaug/test/testA',word_to_index, numeric_naming=False, diagonal_points=True, generate_non_text= True)
process_dataset('../2015/ch4_training_localization_transcription_gt', '../2015/ch4_training_images', '../FY/A-Aug-B-noaug/train/trainB',word_to_index, numeric_naming=False, diagonal_points=False, generate_non_text= False)
process_dataset('../2015/Challenge4_Test_Task1_GT', '../2015/ch4_test_images', '../FY/A-Aug-B-noaug/test/testB',word_to_index, numeric_naming=False,diagonal_points=False, generate_non_text= False)


In [ ]:
from PIL import Image
import os

def process_dataset(data_folder, output_folder, target_size=(64, 64), convert_to_rgb=False):
    images_folder = os.path.join(data_folder, 'images')
    labels_file = os.path.join(data_folder, 'labels.txt')

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'images'), exist_ok=True)

    with open(labels_file, 'r') as labels_f, open(os.path.join(output_folder, 'labels.txt'), 'w') as out_labels_f:
        for line in labels_f:
            idx, label = line.strip().split()
            img_path = os.path.join(images_folder, f'{idx}.png')
            img = Image.open(img_path)

            # Resize every image to the target size
            img = img.resize(target_size)

            if convert_to_rgb:
                img = img.convert('RGB')

            img.save(os.path.join(output_folder, 'images', f'{idx}.png'))
            out_labels_f.write(f'{idx} {label}\n')

# Example usage
base_dir = '../FY/A-Aug-B-noaug/train'
process_dataset(os.path.join(base_dir, 'trainA'), '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/trainA', target_size=(128, 128), convert_to_rgb=True)
process_dataset(os.path.join(base_dir, 'trainB'), '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/trainB', target_size=(128, 128), convert_to_rgb=True)

# Example usage
base_dir = '../FY/A-Aug-B-noaug/test'
process_dataset(os.path.join(base_dir, 'testA'), '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/testA', target_size=(128, 128), convert_to_rgb=True)
process_dataset(os.path.join(base_dir, 'testB'), '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/testB', target_size=(128, 128), convert_to_rgb=True)

In [ ]:
import shutil
import os

source_folder = '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/trainA'
destination_folder = '../FY/icdar2013_A-Aug-B-noaug-ds/train'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/trainB'
destination_folder = '../FY/icdar2015_A-Aug-B-noaug-ds/train'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/testA'
destination_folder = '../FY/icdar2013_A-Aug-B-noaug-ds/test'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")

source_folder = '../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/testB'
destination_folder = '../FY/icdar2015_A-Aug-B-noaug-ds/test'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print(f"Folder copied from {source_folder} to {destination_folder}")


In [ ]:
!python ../cyclegan/pytorch-CycleGAN-and-pix2pix/train.py --name test_svhn2mnist \
--dataroot ../x/jhoffman/cyclegan_data/svhn2mnist/ --resize_or_crop=None \
--loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
--no_flip --model cycle_gan --lambda_A 1 --lambda_B 1 --lambda_identity 1.0 --gpu_ids 0

In [ ]:
import os

#A Source-only model that is trained with the labelled source data and evaluated over the target data
!python ../cyclegan/pytorch-CycleGAN-and-pix2pix/train.py --name ICDAR2013to2015_noIdentity \
--dataroot ../FY/cyclegan_data/ICDAR2013_2015/ --resize_or_crop=None \
--loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
--no_flip --model cycle_gan --lambda_A 1 --lambda_B 1 --lambda_identity 0 --gpu_ids 0 --save_epoch_freq 1 

In [ ]:

#A Source-only model that is trained with the labelled source data and evaluated over the target data
!python ../cyclegan/pytorch-CycleGAN-and-pix2pix/train.py --name ICDAR2013to2015__A-Aug-B-noaug \
--dataroot ../FY/cyclegan_data/ICDAR2013_2015_A-Aug-B-noaug/ --resize_or_crop=None \
--loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
--no_flip --model cycle_gan --lambda_A 1 --lambda_B 1 --lambda_identity 0 --gpu_ids 0 --save_epoch_freq 1 --niter_decay 100 --save_latest_freq 50

In [ ]:
#CUDA_VISIBLE_DEVICES=1 
!python cyclegan/pytorch-CycleGAN-and-pix2pix/train.py --name cycada_icdar132icdar15_noIdentity_semantic \
    --resize_or_crop=None \
    --loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
    --model cycle_gan_semantic \
    --lambda_A 1 --lambda_B 1 --lambda_identity 0 \
    --no_flip --batchSize 100 \
    --dataset_mode icdar2013_icdar2015 --dataroot FY/ \
    --which_direction AtoB --gpu_ids -1

In [ ]:
%cd ..
!ls

In [ ]:
!chmod +x cyclegan/pytorch-CycleGAN-and-pix2pix/test_icdar2013to2015.sh


In [ ]:
#!/bin/bash


In [12]:
!python ..\cyclegan\pytorch-CycleGAN-and-pix2pix\test.py --name ICDAR2013to2015__A-Aug-B-noaug \
    --resize_or_crop=None \
    --loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
    --model cycle_gan \
    --no_flip --batchSize 100 \
    --dataroot ..\FY\cyclegan_data\ICDAR2013_2015_A-Aug-B-noaug \
    --which_direction AtoB \
    --phase train \
    --how_many 2500 \
    --which_epoch "epoch 1 steps 1900" --gpu_ids 0

------------ Options -------------
aspect_ratio: 1.0
batchSize: 100
checkpoints_dir: ./checkpoints
dataroot: ..\FY\cyclegan_data\ICDAR2013_2015_A-Aug-B-noaug
dataset_mode: unaligned
display_id: 1
display_port: 8097
display_server: http://localhost
display_winsize: 256
fineSize: 32
gpu_ids: [0]
how_many: 2500
init_type: normal
input_nc: 3
isTrain: False
loadSize: 32
max_dataset_size: inf
model: cycle_gan
nThreads: 4
n_layers_D: 3
name: ICDAR2013to2015__A-Aug-B-noaug
ndf: 64
ngf: 64
no_dropout: False
no_flip: True
norm: instance
ntest: inf
output_nc: 3
phase: train
resize_or_crop: None
results_dir: ./results/
serial_batches: False
suffix: 
verbose: False
which_direction: AtoB
which_epoch: epoch 1 steps 1900
which_model_netD: n_layers
which_model_netG: resnet_9blocks
-------------- End ----------------
CustomDatasetDataLoader
dataset [UnalignedDataset] was created
Batch 1
Domain A Images Tensor Shape: torch.Size([1, 3, 128, 128])
Domain B Images Tensor Shape: torch.Size([1, 3, 128, 128])


In [ ]:
!python ..\cyclegan\pytorch-CycleGAN-and-pix2pix\test.py --name ICDAR2013to2015__A-Aug-B-noaug \
    --resize_or_crop=None \
    --loadSize=32 --fineSize=32 --which_model_netD n_layers --n_layers_D 3 \
    --model cycle_gan \
    --no_flip --batchSize 100 \
    --dataroot ..\FY\cyclegan_data\ICDAR2013_2015_A-Aug-B-noaug \
    --which_direction AtoB \
    --phase test \
    --how_many 2500 \
    --which_epoch "epoch 1 steps 1900" --gpu_ids 0

In [15]:
import os
import shutil
# 后跑
# Source and destination directories
src_dir = '../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/'
dest_dir = '../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images'

# Create the destination directory if it does not exist
os.makedirs(dest_dir, exist_ok=True)

# Iterate through the files in the source directory
for filename in sorted(os.listdir(src_dir)):
    # Check if the file is a 'fake_B' image
    if filename.endswith('fake_B.png'):
        src_path = os.path.join(src_dir, filename)

        # Extract the leading number from the filename (e.g., '106' from '106_fake_B.png')
        file_number = filename.split('_')[0]

        # Define the destination path for the image
        dest_path = os.path.join(dest_dir, f'{file_number}.png')

        # Copy the image to the destination
        shutil.copyfile(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")

print("All 'fake_B' images have been copied and renamed.")

source_file = '../FY/icdar2013/train/labels.txt'
destination_folder = '../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Specify the destination file path
destination_file = os.path.join(destination_folder, 'labels.txt')

# Copy the file
shutil.copyfile(source_file, destination_file)

print(f"File copied from {source_file} to {destination_file}")

Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/0_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\0.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1000_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\1000.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1001_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\1001.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1002_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\1002.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1003_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\1003.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1004_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/train/images\1004.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/train_1900/images/1

In [16]:
import os
import shutil

# Source and destination directories
src_dir = '../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/'
dest_dir = '../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images'


# Create the destination directory if it does not exist
os.makedirs(dest_dir, exist_ok=True)

# Iterate through the files in the source directory
for filename in sorted(os.listdir(src_dir)):
    # Check if the file is a 'fake_B' image
    if filename.endswith('fake_B.png'):
        src_path = os.path.join(src_dir, filename)

        # Extract the leading number from the filename (e.g., '106' from '106_fake_B.png')
        file_number = filename.split('_')[0]

        # Define the destination path for the image
        dest_path = os.path.join(dest_dir, f'{file_number}.png')

        # Copy the image to the destination
        shutil.copyfile(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")

print("All 'fake_B' images have been copied and renamed.")

source_file = '../FY/icdar2013/test/labels.txt'
destination_folder = '../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Specify the destination file path
destination_file = os.path.join(destination_folder, 'labels.txt')

# Copy the file
shutil.copyfile(source_file, destination_file)

print(f"File copied from {source_file} to {destination_file}")

Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/0_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\0.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1000_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\1000.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1001_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\1001.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1002_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\1002.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1003_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\1003.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1004_fake_B.png to ../FY/icdar2013to2015_A-Aug-B-noaug-ds/test/images\1004.png
Copied ../scripts/results/ICDAR2013to2015__A-Aug-B-noaug/test_1900/images/1005_fake_B.pn

In [ ]:
import subprocess

# Path to the Bash script
script_path = 'cyclegan/pytorch-CycleGAN-and-pix2pix/test_icdar2013to2015.sh'

# Run the Bash script
result = subprocess.run([script_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output and error (if any)
print("Output:", result.stdout)
print("Error:", result.stderr)


In [ ]:
# A Source-only model that is trained with the labelled source data and evaluated over the target data;
! python train_adda-A-Aug-B-noaug-adaptiiveUDA.py


In [ ]:
# A Source-only model that is trained with the labelled source data and evaluated over the target data;
! python train_adda_A-Aug-B-noaug-sourceonly.py 

In [ ]:
! python train_adda-A-Aug-B-noaug-adaptiiveUDA-epoch6.py